# Libraries

In [4]:
import joblib
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Prepare Data

In [5]:
# Load from pkl
xall, yall, harai = joblib.load("./data.pkl")

In [6]:
# Input features
xall

,raceid,horseid,racedate,futan,umaban,wakuban,blinker,age,bataijyu,zogen,...,grade_5,wintime_5,lap_s3_5,lap_s4_5,lap_l3_5,lap_l4_5,f,ff,m,mf
0,2020010506010101,2017103291,20200105,54.0,1,1,0,3.0,432.0,4.0,...,None,NaN,NaN,NaN,NaN,NaN,1.120002e+09,1.120002e+09,1.220057e+09,1.120002e+09
1,2020010506010101,2017101861,20200105,54.0,2,1,0,3.0,424.0,-4.0,...,None,NaN,NaN,NaN,NaN,NaN,1.120002e+09,1.140004e+09,1.220063e+09,1.120002e+09
2,2020010506010101,2017103186,20200105,54.0,3,2,0,3.0,458.0,0.0,...,None,NaN,NaN,NaN,NaN,NaN,1.120002e+09,1.120002e+09,1.220061e+09,1.140006e+09
3,2020010506010101,2017102095,20200105,53.0,4,2,0,3.0,464.0,10.0,...,None,NaN,NaN,NaN,NaN,NaN,1.120002e+09,1.120002e+09,1.220062e+09,1.120002e+09
4,2020010506010101,2017103287,20200105,52.0,5,3,1,3.0,460.0,-2.0,...,None,NaN,NaN,NaN,NaN,NaN,1.120002e+09,1.140006e+09,1.220062e+09,1.140005e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143318,2022122809060912,2018105223,20221228,53.0,12,6,0,4.0,452.0,0.0,...,E,69.4,33.6,45.3,35.8,46.9,1.140007e+09,1.140005e+09,1.220065e+09,1.140006e+09
143319,2022122809060912,2017106137,20221228,56.0,13,7,0,5.0,492.0,6.0,...,E,67.3,32.8,44.1,34.5,45.4,1.120002e+09,1.140004e+09,1.220050e+09,1.120002e+09
143320,2022122809060912,2017100461,20221228,56.0,14,7,0,5.0,504.0,4.0,...,E,80.4,35.6,47.1,33.3,44.8,1.120002e+09,1.140004e+09,1.220055e+09,1.120002e+09
143321,2022122809060912,2018106389,20221228,52.0,15,8,0,4.0,434.0,2.0,...,E,68.4,33.7,45.0,34.7,45.7,1.120002e+09,1.140005e+09,1.220063e+09,1.120002e+09


In [7]:
# # Output feature: finishing position (0: race not finished)
yall

0          2
1         12
2          3
3         11
4          8
          ..
143318    12
143319    16
143320    11
143321     7
143322     8
Name: jyuni, Length: 143323, dtype: int64

In [8]:
# Payout
harai

,RaceID,TorokuTosu,SyussoTosu,FuseirituFlag,TokubaraiFlag,HenkanFlag,HenkanUma,HenkanWaku,HenkanDoWaku,PayTansyo,...,PayReserved1,PayUmatan,PaySanrenpuku,PaySanrentan,Year,MonthDay,JyoCD,Kaiji,Nichiji,RaceNum
0,2020010606010203,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '09', 'Pay': '000000200', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0913', 'Pay': '000000510', 'Ninki':...","[{'Kumi': '080913', 'Pay': '000001270', 'Ninki...","[{'Kumi': '091308', 'Pay': '000003840', 'Ninki...",2020,0106,06,01,02,03
1,2022010507010112,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '13', 'Pay': '000001530', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '1315', 'Pay': '000004070', 'Ninki':...","[{'Kumi': '061315', 'Pay': '000002720', 'Ninki...","[{'Kumi': '131506', 'Pay': '000026590', 'Ninki...",2022,0105,07,01,01,12
2,2020010506010101,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '07', 'Pay': '000000360', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0701', 'Pay': '000003650', 'Ninki':...","[{'Kumi': '010307', 'Pay': '000098210', 'Ninki...","[{'Kumi': '070103', 'Pay': '000280650', 'Ninki...",2020,0105,06,01,01,01
3,2020010606010204,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '12', 'Pay': '000000280', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '1211', 'Pay': '000003040', 'Ninki':...","[{'Kumi': '031112', 'Pay': '000000920', 'Ninki...","[{'Kumi': '121103', 'Pay': '000008340', 'Ninki...",2020,0106,06,01,02,04
4,2020010606010205,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '07', 'Pay': '000000380', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0716', 'Pay': '000001730', 'Ninki':...","[{'Kumi': '070916', 'Pay': '000002130', 'Ninki...","[{'Kumi': '071609', 'Pay': '000009690', 'Ninki...",2020,0106,06,01,02,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10363,2022122806050903,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '01', 'Pay': '000000600', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0107', 'Pay': '000004600', 'Ninki':...","[{'Kumi': '010307', 'Pay': '000023600', 'Ninki...","[{'Kumi': '010703', 'Pay': '000094870', 'Ninki...",2022,1228,06,05,09,03
10364,2022122806050902,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '04', 'Pay': '000000750', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0406', 'Pay': '000030980', 'Ninki':...","[{'Kumi': '040611', 'Pay': '000013110', 'Ninki...","[{'Kumi': '040611', 'Pay': '000138570', 'Ninki...",2022,1228,06,05,09,02
10365,2022122809060911,13,13,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]

In [9]:
'''
Preprocess data
'''

# Convert string values to float
for col in xall.columns:
    if xall[col].dtype == "object":
        # Convert to numeric where possible
        xall[col] = pd.to_numeric(xall[col], errors='coerce')
        # Replace NaN (from non-numeric values) with 0
        xall[col] = xall[col].fillna(0)
        print(f"Converted column: {col}")

Converted column: KisyuCode
Converted column: TozaiCD
Converted column: ChokyosiCode
Converted column: BanusiCode
Converted column: course
Converted column: cls
Converted column: grade
Converted column: BreederCode
Converted column: KisyuCode_1
Converted column: course_1
Converted column: cls_1
Converted column: grade_1
Converted column: KisyuCode_2
Converted column: course_2
Converted column: cls_2
Converted column: grade_2
Converted column: KisyuCode_3
Converted column: course_3
Converted column: cls_3
Converted column: grade_3
Converted column: KisyuCode_4
Converted column: course_4
Converted column: cls_4
Converted column: grade_4
Converted column: KisyuCode_5
Converted column: course_5
Converted column: cls_5
Converted column: grade_5


In [10]:
'''
Split into Train-Test data
'''
# Number of days to use of testing
TEST_DAYS = 90

# Date at which the data splits
rd = xall["racedate"].unique()[-TEST_DAYS] 

# split data
xtrain = xall[xall['racedate']<rd].reset_index(drop=True).copy()
ytrain = yall[xtrain.index].reset_index(drop=True).copy()
xtest = xall[xall['racedate']>=rd].reset_index(drop=True).copy()
ytest = yall[xtest.index].reset_index(drop=True).copy()

In [11]:
print(xtrain.shape, ytrain.shape, xtest.shape, ytest.shape)

(104436, 224) (104436,) (38887, 224) (38887,)


# Winner prediction

## Models

In [12]:
model_prediction_winner = {}

### Light Gradient Boosting Machine (Binary)

In [13]:
# Binarize outputs (simplify ranking to win / lose)
ytrain_lgb = ytrain.apply(lambda x: 1 if x==1 else 0)
ytest_lgb = ytest.apply(lambda x: 1 if x==1 else 0)

In [14]:
# Training parameters
num_boost_round = 100
early_stopping_round = 10
lgb_train_parameters = {
 'max_depth': 10,
 'min_data_in_leaf': 50,
 'learning_rate': 0.01,
 'seed': 1,
 'objective': 'binary',
 'metric': 'binary_logloss',
}

In [15]:
# Validation data (training_data=validation_data）
xtrain_val = xtrain.copy()
ytrain_val_lgb = ytrain_lgb.copy()
xtest_val = xtrain.copy()
ytest_val_lgb = ytrain_lgb.copy()

In [16]:
# Prepare dataset for LGB model
train_data_lgb = lgb.Dataset(xtrain_val, label=ytrain_val_lgb) 
valid_data_lgb = lgb.Dataset(xtest_val, label=ytest_val_lgb)

In [17]:
# Train a LGB model on our dataset
gbm = lgb.train(lgb_train_parameters, 
                train_data_lgb,
                valid_sets=[valid_data_lgb],
                num_boost_round=num_boost_round,
                callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_round, 
                            verbose=1)]
               )

[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=10) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1024) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=10) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1024) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Number of positive: 7532, number of negative: 96904
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.046413 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 17338
[LightGBM] [Info] Number of data points in the train set: 104

In [18]:
# Predict
ypred = gbm.predict(xtest)
model_prediction_winner['LightGBM'] = ypred

### XGBoost (Binary)

In [19]:
# XGBoost Official Defaults Baseline
xgb_train_param = {
    "n_estimators": 100,
    "max_depth": 6,
    "learning_rate": 0.3,
    "objective": 'binary:logistic',
    "random_state": 42  # reproducibility
}

In [20]:
bst = xgb.XGBClassifier(xgb_train_param)
# fit model
bst.fit(xtrain, ytrain)
# make predictions
ypred = bst.predict(xtest)
model_prediction_winner['XGBoost'] = ypred

/home/noah/gakusai/Horse-Racing/.venv/lib/python3.12/site-packages/xgboost/core.py:771: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


### Catboost (Binary)

In [21]:
# Standard Baseline Parameters
cb_train_param = {
    "iterations": 1000,         # Standard default
    "learning_rate": None,      # Automatically scales based on dataset size
    "depth": 6,                 # The "Goldilocks" depth for CatBoost
    "loss_function": 'Logloss', # Standard for binary
    "bootstrap_type": 'MVS',    # Standard for CPU training
    "verbose": 100              # Standard to see if it's actually learning
}

In [22]:
# 1. Initialize
bst = CatBoostClassifier(**cb_train_param)

# 2. Convert ytrain to: 1 if the horse came in 1st, 0 if they came in any other position
# We do this inline inside the fit function
bst.fit(xtrain, (ytrain == 1).astype(int))

# 3. Predict (This will now output 1 for predicted winners, 0 for others)
ypred = bst.predict(xtest)
model_prediction_winner['CatBoost'] = ypred

Learning rate set to 0.074986
0:	learn: 0.6027906	total: 73.2ms	remaining: 1m 13s
100:	learn: 0.2020550	total: 2.29s	remaining: 20.4s
200:	learn: 0.1972165	total: 4.35s	remaining: 17.3s
300:	learn: 0.1920777	total: 6.53s	remaining: 15.2s
400:	learn: 0.1870432	total: 8.76s	remaining: 13.1s
500:	learn: 0.1822325	total: 11s	remaining: 11s
600:	learn: 0.1779849	total: 13.3s	remaining: 8.81s
700:	learn: 0.1734606	total: 15.6s	remaining: 6.67s
800:	learn: 0.1695032	total: 17.9s	remaining: 4.44s
900:	learn: 0.1656794	total: 20.3s	remaining: 2.23s
999:	learn: 0.1617462	total: 22.7s	remaining: 0us


### Random Forest Classifier

In [23]:
# Random Forest Official Defaults Baseline
rf_train_param = {
    "n_estimators": 100,
    "max_depth": None,      # Traditional RF baseline allows deep trees
    "random_state": 42,     # For reproducible results
    "n_jobs": -1            # Uses all available CPU cores (highly recommended)
}

In [24]:
from sklearn.ensemble import RandomForestClassifier
bst = RandomForestClassifier(**rf_train_param)
bst.fit(xtrain, (ytrain).astype(int))
ypred = bst.predict(xtest)
model_prediction_winner['RandomForest'] = ypred

### Logistic Regression

In [25]:
# Logistic Regression Official Defaults Baseline
logreg_train_param = {
    "penalty": 'l2',
    "C": 1.0,
    "solver": 'lbfgs',
    "max_iter": 100,
    "random_state": 42  # For reproducible results
}

In [26]:
# Fill missing values with 0 (Standard quick fix)
xtrain_filled = xtrain.fillna(0)
xtest_filled = xtest.fillna(0)

# Now use these filled versions to fit
bst = LogisticRegression(**logreg_train_param)
bst.fit(xtrain_filled, (ytrain == 1).astype(int))

ypred = bst.predict(xtest_filled)
model_prediction_winner['LogisticRegression'] = ypred

/home/noah/gakusai/Horse-Racing/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


## Evaluation

### Tansyo

In [27]:
def calc_mrr(ypred, ytrue):
    '''
    Args:
        ypred (numpy.array): predicted ranks (1 or higher) [n, MAX_HORSE]
        ytrue (numpy.array): final confirmed ranks [n, MAX_HORSE]
    
    Returns:
        mrr (int)
    '''
    res = []
    for i in range(ytrue.shape[0]):
        # Which rank did we predict for the horse that finished 1st?
        # If there is a tie, take the smallest value
        indices = np.where(ytrue[i, :] == 1) 
        if indices[0].size > 0:  # There may be races without a 1st-place horse before the start
            res.append(1 / ypred[i, np.min(indices)])
    return np.mean(res)


In [28]:
def get_reports(df_res):
    """
    Summary: 
        Outputs a report of the win predictions.

    Args:
        df_res (pd.DataFrame): Aggregated results from get_win_results

    Returns:
        None

    Note:
        Races with refunds are excluded from the aggregation
    """

    # Hit rate (accuracy)
    tekichu = df_res.apply(lambda x: sum([y in x["true"] for y in x["pred"]]) >= 1, axis=1).astype(int).values
    print("Number of races", df_res.shape[0])
    print("Number of hits", np.sum(tekichu))
    print("Hit rate", np.mean(tekichu), np.std(tekichu, ddof=1), np.std(tekichu, ddof=1) / np.sqrt(df_res.shape[0]))

    # Return rate (profitability)
    modoshi = df_res.apply(lambda x: sum([x["pay"][y] if y in x["true"] else 0 for y in x["pred"]]) + x["henkan"]*100, axis=1).values
    harai = df_res.apply(lambda x: len(x["pred"]), axis=1).values * 100
    print("Total payout", np.sum(modoshi))
    print("Return rate", np.mean(modoshi/harai), np.std(modoshi/harai, ddof=1), np.std(modoshi/harai, ddof=1) / np.sqrt(df_res.shape[0]))
    print("※Mean, standard deviation, standard error")


In [29]:
def get_win_results(df_bet, df_kekka):
    """
    Summary:
        Takes single-win bets as input, merges them with payout data, and outputs the result.

    Args:
        df_bet (pd.DataFrame): Data containing raceid and win predictions
        df_kekka (pd.DataFrame): Payout data extracted from JRA-VAN

    Returns:
        df_results (pd.DataFrame): Aggregated data for each race

    Note:
        Because ties can occur, 'true' is a list and 'pay' is a dict
    """

    # Aggregate by raceid
    res = []
    for raceid in df_bet["raceid"].tolist():
        # Get predictions
        pred = df_bet[df_bet["raceid"]==raceid]["win"].values[0]  # list of predicted horses
        # Check win or loss
        try:
            tmp = df_kekka[df_kekka.raceid==raceid].iloc[0].to_dict()  # target race
        except:
            print("Payout data for this race does not exist.", raceid)
            continue

        # Actual winners (true) and their payouts (pay)
        true = [x["Umaban"] for x in tmp["PayTansyo"] if x["Umaban"] != '  ']  # list because ties possible
        pay = {x["Umaban"]: int(x["Pay"]) for x in tmp["PayTansyo"] if x["Umaban"] != '  '}
        henkan = sum([int(tmp["HenkanUma"][x-1]) for x in pred])  # refunded amounts
        pred = [f"{x:02}" for x in pred]  # convert to string

        res.append({"pred": pred, "true": true, "pay": pay, "henkan": henkan, "raceid": raceid})     

    df_res = pd.DataFrame(res)
    get_reports(df_res)

    return df_res


In [36]:
def evaluate(xtest, ytest, ypred, harai):
    """
    Evaluate predictions for horse races.

    Args:
        xtest (pd.DataFrame): Test data containing race and horse info
        ytest (np.array): True ranks of horses
        ypred (np.array): Predicted scores for horses
        harai (pd.DataFrame): Payout data

    Returns:
        df_bet (pd.DataFrame): Betting predictions per race
        df_res (pd.DataFrame): Aggregated results with payouts
    """

    # 'jyuni': store the true ranks in a matrix of shape (num_races × 18)
    tmp = xtest.copy()
    tmp["jyuni"] = ytest
    tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: x["jyuni"].values)
    jyuni = np.array([[tmp[i][j] if j < len(tmp[i]) else j+1 for j in range(18)] 
                      for i in range(tmp.shape[0])])  # padding for missing horses
    # print("jyuni\n", jyuni)

    # 'pred': store the predicted ranks in a matrix of shape (num_races × 18) 
    # sorted by descending score
    tmp = xtest.copy()
    tmp["pred"] = ypred
    tmp = tmp.groupby("raceid", as_index=False).apply(
        lambda x: np.argsort(np.argsort(-x["pred"].values)) + 1
    )
    pred = np.array([[tmp[i][j] if j < len(tmp[i]) else j+1 for j in range(18)] 
                     for i in range(tmp.shape[0])])  # padding
    #print("pred\n", pred)

    # Compute MRR (Mean Reciprocal Rank)
    mrr = calc_mrr(pred, jyuni)
    print("mrr\n", mrr)
        
    # df_bet: extract unique raceid rows
    df_bet = xtest[["raceid"]].groupby("raceid", as_index=False).apply(lambda x: x.iloc[0])

    # win: buy the horse ranked #1 by predicted score (single win)
    kaime = [[np.where(pred[i, :] == 1)[0][0] + 1] for i in range(pred.shape[0])]
    df_bet["win"] = kaime

    # Align payout data
    harai = harai.copy()
    harai.columns = [x.replace("RaceID", "raceid") for x in harai.columns]
    harai.raceid = harai.raceid.astype(int)

    # Compute and display results
    df_res = get_win_results(df_bet, harai)

    return df_bet, df_res


In [37]:
# Evaluate every model
results_winner_tansyo = {}

for model_name, model_ypred in model_prediction_winner.items():
    print(f"---- Evaluating {model_name} ----")
    df_bet, df_res = evaluate(xtest, yall[xtest.index].reset_index(drop=True).copy(), model_ypred, harai)
    results_winner_tansyo[model_name] = {"df_bet": df_bet, "df_res": df_res}

---- Evaluating LightGBM ----


/tmp/ipykernel_11403/2116485817.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: x["jyuni"].values)
/tmp/ipykernel_11403/2116485817.py:28: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(


mrr
 0.23596079440398804


/tmp/ipykernel_11403/2116485817.py:40: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_bet = xtest[["raceid"]].groupby("raceid", as_index=False).apply(lambda x: x.iloc[0])


Number of races 2856
Number of hits 976
Hit rate 0.34173669467787116 0.4743748715278793 0.008876516691665327
Total payout 235830
Return rate 0.8257352941176471 1.2432512677833703 0.023263754664889724
※Mean, standard deviation, standard error
---- Evaluating XGBoost ----


...

Number of races 2856
Number of hits 28
Hit rate 0.00980392156862745 0.0985454462284114 0.0018439853180180364
Total payout 190020
Return rate 0.6653361344537815 8.966093130041159 0.16777380106896903
※Mean, standard deviation, standard error
---- Evaluating LogisticRegression ----


/tmp/ipykernel_11403/2116485817.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: x["jyuni"].values)
/tmp/ipykernel_11403/2116485817.py:28: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(


mrr
 0.2693263591562004


/tmp/ipykernel_11403/2116485817.py:40: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_bet = xtest[["raceid"]].groupby("raceid", as_index=False).apply(lambda x: x.iloc[0])


Number of races 2856
Number of hits 213
Hit rate 0.07457983193277311 0.2627581681329854 0.004916738649734262
Total payout 191200
Return rate 0.6694677871148459 5.183416828152404 0.09699224970909885
※Mean, standard deviation, standard error


### Fukusyo

In [32]:
def calc_mrr_fukusyo(ypred, ytrue):
    res = []
    nb_races = ytrue.shape[0]

    for i in range(nb_races):
        chosen = np.argmin(ypred[i, :])   # rank 1
        if ytrue[i, chosen] <= 3:
            res.append(1.0)
        else:
            res.append(0.0)

    return np.mean(res)

def get_reports_fukusyo(df_res):
    # Hit rate (accuracy)
    tekichu = df_res.apply(lambda x: sum([y in x["true"] for y in x["pred"]]) >= 1, axis=1).astype(int).values
    print("Number of races", df_res.shape[0])
    print("Number of hits", np.sum(tekichu))
    print("Hit rate", np.mean(tekichu), np.std(tekichu, ddof=1), np.std(tekichu, ddof=1) / np.sqrt(df_res.shape[0]))

    # Return rate (profitability)
    modoshi = df_res.apply(lambda x: sum([x["pay"][y] if y in x["true"] else 0 for y in x["pred"]]) + x["henkan"]*100, axis=1).values
    harai = df_res.apply(lambda x: len(x["pred"]), axis=1).values * 100
    print("Total payout", np.sum(modoshi))
    print("Return rate", np.mean(modoshi/harai), np.std(modoshi/harai, ddof=1), np.std(modoshi/harai, ddof=1) / np.sqrt(df_res.shape[0]))
    print("※Mean, standard deviation, standard error")

def get_win_results_fukusyo(df_bet, df_kekka):
    # Aggregate by raceid
    res = []
    for raceid in df_bet["raceid"].tolist():
        # Get predictions
        pred = df_bet[df_bet["raceid"]==raceid]["top3"].values[0]  # list of predicted horses
        # Check bets
        try:
            tmp = df_kekka[df_kekka.raceid==raceid].iloc[0].to_dict()  # target race
        except:
            print("Payout data for this race does not exist.", raceid)
            continue

        # Actual winners (true) and their payouts (pay)
        true = [x["Umaban"] for x in tmp["PayFukusyo"] if x["Umaban"] != '  ']  # list because ties possible
        pay = {
            x["Umaban"]: int(x["Pay"])
            for x in tmp["PayFukusyo"]
            if x["Umaban"].strip() != "" and x["Pay"].strip() != ""
        }

        henkan = sum(
            int(tmp["HenkanUma"][x-1])
            for x in pred
            if str(tmp["HenkanUma"][x-1]).strip() != ""
        )

        pred = [f"{x:02}" for x in pred]  # convert to string

        res.append({"pred": pred, "true": true, "pay": pay, "henkan": henkan, "raceid": raceid})     

    df_res = pd.DataFrame(res)
    get_reports_fukusyo(df_res)

    return df_res

def evaluate_fukusyo(xtest, ytest, ypred, harai):
    # 'jyuni': store the true ranks in a matrix of shape (num_races × 18)
    tmp = xtest.copy()
    tmp["jyuni"] = ytest
    tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: x["jyuni"].values)
    jyuni = np.array([[tmp[i][j] if j < len(tmp[i]) else j+1 for j in range(18)] 
                      for i in range(tmp.shape[0])])  # padding for missing horses

    # 'pred': store the predicted ranks in a matrix of shape (num_races × 18) 
    # sorted by descending score
    tmp = xtest.copy()
    tmp["pred"] = ypred
    tmp = tmp.groupby("raceid", as_index=False).apply(
        lambda x: np.argsort(np.argsort(-x["pred"].values)) + 1
    )
    pred = np.array([[tmp[i][j] if j < len(tmp[i]) else j+1 for j in range(18)] 
                     for i in range(tmp.shape[0])])  # padding
    
    # Compute MRR (Mean Reciprocal Rank)
    mrr = calc_mrr_fukusyo(pred, jyuni)
    print("mrr\n", mrr)
        
    # df_bet: extract unique raceid rows
    df_bet = xtest[["raceid"]].groupby("raceid", as_index=False).apply(lambda x: x.iloc[0])

    # win: buy the horse ranked <= 3 by predicted score
    kaime = [[np.where(pred[i, :] <= 3)[0][0] + 1] for i in range(pred.shape[0])]
    df_bet["top3"] = kaime

    # Align payout data
    harai = harai.copy()
    harai.columns = [x.replace("RaceID", "raceid") for x in harai.columns]
    harai.raceid = harai.raceid.astype(int)

    # Compute and display results
    df_res = get_win_results_fukusyo(df_bet, harai)

    return df_bet, df_res


In [33]:
# Evaluate every model
results_winner_fukusyo = {}

for model_name, model_ypred in model_prediction_winner.items():
    print(f"---- Evaluating {model_name} ----")
    df_bet, df_res = evaluate_fukusyo(xtest, yall[xtest.index].reset_index(drop=True).copy(), model_ypred, harai)
    results_winner_fukusyo[model_name] = {"df_bet": df_bet, "df_res": df_res}

---- Evaluating LightGBM ----


/tmp/ipykernel_11403/4068334179.py:68: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: x["jyuni"].values)
/tmp/ipykernel_11403/4068334179.py:76: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(
/tmp/ipykernel_11403/4068334179.py:87: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This beha

mrr
 0.21498599439775912
Number of races 2856
Number of hits 1495
Hit rate 0.5234593837535014 0.4995368156839375 0.009347347738378716
Total payout 237530
Return rate 0.831687675070028 0.8819780081142924 0.01650359869504206
※Mean, standard deviation, standard error
---- Evaluating XGBoost ----


/tmp/ipykernel_11403/4068334179.py:68: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: x["jyuni"].values)
/tmp/ipykernel_11403/4068334179.py:76: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(


mrr
 0.22303921568627452


...

Number of races 2856
Number of hits 166
Hit rate 0.05812324929971989 0.23401733333219402 0.0043789392949330095
Total payout 159090
Return rate 0.5570378151260504 3.3229593788136387 0.062179314635201106
※Mean, standard deviation, standard error
---- Evaluating LogisticRegression ----


/tmp/ipykernel_11403/4068334179.py:68: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: x["jyuni"].values)
/tmp/ipykernel_11403/4068334179.py:76: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(


mrr
 0.22023809523809523


/tmp/ipykernel_11403/4068334179.py:87: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_bet = xtest[["raceid"]].groupby("raceid", as_index=False).apply(lambda x: x.iloc[0])


Number of races 2856
Number of hits 612
Hit rate 0.21428571428571427 0.4103977579614743 0.007679375041586616
Total payout 182030
Return rate 0.6373599439775911 1.853926934024014 0.03469073587727956
※Mean, standard deviation, standard error


### Umaren

In [40]:
def calc_mrr_umaren(ypred, ytrue):
    res = []
    for i in range(ypred.shape[0]):
        top2 = np.argsort(ypred[i])[:2]
        if np.all(ytrue[i, top2] <= 2):
            res.append(1.0)
        else:
            res.append(0.0)
    return np.mean(res)


def get_reports_umaren(df_res):
    tekichu = df_res["hit"].astype(int).values
    print("Number of races", df_res.shape[0])
    print("Number of hits", np.sum(tekichu))
    print(
        "Hit rate",
        np.mean(tekichu),
        np.std(tekichu, ddof=1),
        np.std(tekichu, ddof=1) / np.sqrt(df_res.shape[0])
    )

    modoshi = df_res["payout"].values
    harai = np.ones(len(modoshi)) * 100
    print("Total payout", np.sum(modoshi))
    print(
        "Return rate",
        np.mean(modoshi / harai),
        np.std(modoshi / harai, ddof=1),
        np.std(modoshi / harai, ddof=1) / np.sqrt(df_res.shape[0])
    )
    print("※Mean, standard deviation, standard error")


def get_win_results_umaren(df_bet, df_kekka):
    res = []

    for raceid in df_bet["raceid"].tolist():
        pred = df_bet[df_bet["raceid"] == raceid]["top2"].values[0]

        try:
            tmp = df_kekka[df_kekka.raceid == raceid].iloc[0].to_dict()
        except:
            continue

        true_pairs = [
            frozenset([x["Kumi"][:2], x["Kumi"][2:]])
            for x in tmp["PayUmaren"]
            if x["Kumi"].strip() != ""
        ]

        pay = {
            frozenset([x["Kumi"][:2], x["Kumi"][2:]]): int(x["Pay"])
            for x in tmp["PayUmaren"]
            if x["Kumi"].strip() != "" and x["Pay"].strip() != ""
        }


        henkan = sum(
            int(tmp["HenkanUma"][x - 1])
            for x in pred
            if str(tmp["HenkanUma"][x - 1]).strip() != ""
        )

        pred_pair = frozenset([f"{pred[0]:02}", f"{pred[1]:02}"])

        hit = pred_pair in pay
        payout = pay[pred_pair] + henkan * 100 if hit else henkan * 100

        res.append({
            "raceid": raceid,
            "pred": pred_pair,
            "true": true_pairs,
            "hit": hit,
            "payout": payout
        })

    df_res = pd.DataFrame(res)
    get_reports_umaren(df_res)
    return df_res


def evaluate_umaren(xtest, ytest, ypred, harai):
    tmp = xtest.copy()
    tmp["jyuni"] = ytest
    tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: x["jyuni"].values)
    jyuni = np.array([
        [tmp[i][j] if j < len(tmp[i]) else j + 1 for j in range(18)]
        for i in range(tmp.shape[0])
    ])

    tmp = xtest.copy()
    tmp["pred"] = ypred
    tmp = tmp.groupby("raceid", as_index=False).apply(
        lambda x: np.argsort(np.argsort(-x["pred"].values)) + 1
    )
    pred = np.array([
        [tmp[i][j] if j < len(tmp[i]) else j + 1 for j in range(18)]
        for i in range(tmp.shape[0])
    ])

    mrr = calc_mrr_umaren(pred, jyuni)
    print("mrr\n", mrr)

    df_bet = xtest[["raceid"]].groupby("raceid", as_index=False).apply(lambda x: x.iloc[0])

    kaime = [list(np.argsort(pred[i])[:2] + 1) for i in range(pred.shape[0])]
    df_bet["top2"] = kaime

    harai = harai.copy()
    harai.columns = [c.replace("RaceID", "raceid") for c in harai.columns]
    harai.raceid = harai.raceid.astype(int)

    df_res = get_win_results_umaren(df_bet, harai)
    return df_bet, df_res


In [41]:
# Evaluate every model
results_winner_umaren = {}

for model_name, model_ypred in model_prediction_winner.items():
    print(f"---- Evaluating {model_name} ----")
    df_bet, df_res = evaluate_umaren(xtest, yall[xtest.index].reset_index(drop=True).copy(), model_ypred, harai)
    results_winner_umaren[model_name] = {"df_bet": df_bet, "df_res": df_res}

---- Evaluating LightGBM ----


/tmp/ipykernel_11403/1730945782.py:86: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: x["jyuni"].values)
/tmp/ipykernel_11403/1730945782.py:94: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(


mrr
 0.01365546218487395


/tmp/ipykernel_11403/1730945782.py:105: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_bet = xtest[["raceid"]].groupby("raceid", as_index=False).apply(lambda x: x.iloc[0])


Number of races 2856
Number of hits 428
Hit rate 0.14985994397759103 0.35699659006234774 0.006680130801089021
Total payout 221890
Return rate 0.7769257703081233 2.1537089263632585 0.04030026542569199
※Mean, standard deviation, standard error
---- Evaluating XGBoost ----


...

Number of races 2856
Number of hits 1
Hit rate 0.00035014005602240897 0.01871202971412799 0.0003501400560224089
Total payout 9300
Return rate 0.032563025210084036 1.7402187634139032 0.03256302521008403
※Mean, standard deviation, standard error
---- Evaluating LogisticRegression ----


/tmp/ipykernel_11403/1730945782.py:86: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: x["jyuni"].values)
/tmp/ipykernel_11403/1730945782.py:94: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = tmp.groupby("raceid", as_index=False).apply(


mrr
 0.01715686274509804


/tmp/ipykernel_11403/1730945782.py:105: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_bet = xtest[["raceid"]].groupby("raceid", as_index=False).apply(lambda x: x.iloc[0])


Number of races 2856
Number of hits 51
Hit rate 0.017857142857142856 0.13245530682547105 0.0024785076371121547
Total payout 154420
Return rate 0.5406862745098039 5.630792631456031 0.10536355903389821
※Mean, standard deviation, standard error


# Prediction - Ranking

## Models

In [ ]:
model_prediction_ranking = {}

### LightGBM (rank:pair-wise)

### XGBoost (rank:pair-wise)

### Catboost (rank:pair-wise)

### Random Forest Classifier

## Evaluation

### Tansyo

### Fukusyo

### Wakuren

### Umaren

### Wide

# Prediction - Time

## Models

In [ ]:
model_prediction_time = {}

### LightGBM Regressor

### XGBoost Regressor

### Catboost Regressor

### Neural Network Regressor

## Evaluation

### Tansyo 

### Fukusyo

### Wakuren

### Umaren

### Wide

# Results - Methods comparison